<h1 style="text-align: center;">Release Notes Agent - DevOps to Confluence</h1>
<p style="text-align: center;"><em>Powered by Google ADK, Azure DevOps MCP & Atlassian MCP</em></p>

**This agent automates release notes creation from Azure DevOps user stories and publishes them to Confluence.**

### How It Works

1. **Fetch Tickets** - Retrieves work items from a configured Azure DevOps query
2. **Extract Data** - Gets ticket details: ID, Title, Description, Release Date, Story Owner, Area, Customer, URL
3. **Generate Release Notes** - Creates user-friendly summaries from technical descriptions
4. **Save to Excel** - Stores all data incrementally in an Excel file (prevents data loss)
5. **Publish to Confluence** - Creates a formatted Markdown page under the specified parent page

### Architecture
```
┌─────────────────────────────────────────────────────────────┐
│                    SequentialAgent                          │
│                  (ReleaseNotesWorkflow)                     │
├─────────────────────────┬───────────────────────────────────┤
│  TicketProcessorAgent   │     ConfluencePublisherAgent      │
│  ┌───────────────────┐  │     ┌───────────────────────┐     │
│  │ Azure DevOps MCP  │  │     │ read_excel_data       │     │
│  │ initialize_excel  │  │────▶│ format_confluence     │     │
│  │ append_ticket     │  │     │ Atlassian MCP         │     │
│  └───────────────────┘  │     └───────────────────────┘     │
└─────────────────────────┴───────────────────────────────────┘
```

### Prerequisites

- **Python 3.10+** with virtual environment
- **Node.js 18+** - Required for MCP servers (npx)
- **Azure DevOps Access** - Browser authentication triggered on first run
- **Atlassian Cloud Access** - Browser authentication triggered on first run
- **Environment Variables** - `GOOGLE_API_KEY` in `.env` file

### Configuration

Update these variables in the notebook to match your setup:

| Variable | Description |
|----------|-------------|
| `AZURE_DEVOPS_ORG` | Your Azure DevOps organization name |
| `AZURE_DEVOPS_PROJECT` | Project containing work items |
| `AZURE_DEVOPS_QUERY_ID` | Saved query ID returning tickets for release notes |
| `MODEL_NAME` | Gemini model to use |
| `spaceId` | Confluence space ID (in agent instruction) |
| `parentPageId` | Parent page ID for release notes (in agent instruction) |

### Usage
```python
await chat()
# Then type: "Create release notes and publish to Confluence"
```

In [ ]:
%pip install google-adk mcp openpyxl python-dotenv

In [8]:
import os
from datetime import datetime
from typing import List
import contextlib
import warnings

from dotenv import load_dotenv
from openpyxl import Workbook, load_workbook
from google.genai import types
from google.adk.agents import LlmAgent, SequentialAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset, StdioConnectionParams, StdioServerParameters

In [9]:
load_dotenv()

AZURE_DEVOPS_ORG = "UPPER-Network"
AZURE_DEVOPS_PROJECT = "MARMIND"
AZURE_DEVOPS_QUERY_ID = "41ec302e-5520-4a2b-a0ef-0a0f3de37f25"  

# MODEL_NAME = "gemini-2.5-flash"
MODEL_NAME = "gemini-3-flash-preview"
OUTPUT_DIR = "./output"

GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY")

if GEMINI_API_KEY:
    print("Google API key loaded!")
else:
    print("Google API key missing!")

Google API key loaded!


### Release Notes Generation Instructions 

In [10]:
RELEASE_NOTES_INSTRUCTIONS = """
## Guidelines for Creating Release Notes

### Format
- Write in clear, concise language that end-users can understand
- Keep each release note to 2-3 sentences maximum
- Start with an action verb (Added, Fixed, Improved, Updated, etc.)

### Content Guidelines
- Focus on what is added/changed/fixed and the USER BENEFIT, not the technical implementation
- Remove internal jargon and technical details
- Highlight what was added or changed and why it matters to users

### What to Include
- New features or functionality
- Improvements to existing features
- Bug fixes (describe what was fixed)
- UI/UX changes

### What to Exclude
- Internal ticket information
- Developer-focused technical details
- Code changes or implementation specifics

### Example
TICKET DESCRIPTION:
"Implemented caching layer for user profile API calls to reduce database load.
Added Redis integration with 5-minute TTL."

RELEASE NOTE:
"Improved: User profile pages now load faster due to optimized data retrieval."
"""

### Agent Local Tools

In [11]:
async def initialize_excel() -> dict:
    """
    Create a new Excel file with headers for storing release notes data.
    
    Creates the output directory if it doesn't exist and initializes an Excel
    workbook with predefined column headers. The filename includes the current
    date for easy identification.
    
    Returns:
        dict: Status and file path, e.g., {"status": "success", "file_path": "./output/release_notes_2026-01-01.xlsx"}
    """
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    date_str = datetime.now().strftime("%Y-%m-%d")
    filename = f"release_notes_{date_str}.xlsx"
    filepath = os.path.join(OUTPUT_DIR, filename)
    
    wb = Workbook()
    ws = wb.active
    ws.title = "Release Notes"
    
    headers = ["ID", "Title", "Description", "Release Date", "Story Owner", "Area", "Customer", "Ticket URL", "Release Notes", "Release Notes Summary"]
    for col, header in enumerate(headers, 1):
        ws.cell(row=1, column=col, value=header)
    
    wb.save(filepath)
    return {"status": "success", "file_path": filepath}


async def append_ticket_to_excel(
    file_path: str,
    ticket_id: str,
    title: str,
    description: str,
    release_date: str,
    story_owner: str,
    area: str,
    customer: str,
    ticket_url: str,
    release_note: str,
    release_notes_summary: str
) -> dict:
    """
    Append a single ticket to the Excel file. Call this for EACH ticket after getting its details.

    Returns:
        dict: Status with file path and ticket ID, or error message if file not found
    """
    if not os.path.exists(file_path):
        return {"status": "error", "message": f"File not found: {file_path}"}
    
    wb = load_workbook(file_path)
    ws = wb.active
    
    next_row = ws.max_row + 1
    ws.cell(row=next_row, column=1, value=ticket_id)
    ws.cell(row=next_row, column=2, value=title)
    ws.cell(row=next_row, column=3, value=description)
    ws.cell(row=next_row, column=4, value=release_date)
    ws.cell(row=next_row, column=5, value=story_owner)
    ws.cell(row=next_row, column=6, value=area)
    ws.cell(row=next_row, column=7, value=customer)
    ws.cell(row=next_row, column=8, value=ticket_url)
    ws.cell(row=next_row, column=9, value=release_note)
    ws.cell(row=next_row, column=10, value=release_notes_summary)
    
    wb.save(file_path)
    return {"status": "success", "file_path": file_path, "ticket_id": ticket_id}


async def read_excel_data(file_path: str) -> dict:
    """
    Read all release notes data from the Excel file.
    
    Parses the Excel file created by the TicketProcessorAgent and returns
    structured data ready for Confluence publishing.
    
    Args:
        file_path: Path to the Excel file containing release notes data
    
    Returns:
        dict: Contains status, release_date (from first ticket), and list of ticket dicts,
              or error message if file not found
    """
    if not os.path.exists(file_path):
        return {"status": "error", "message": f"File not found: {file_path}"}
    
    wb = load_workbook(file_path)
    ws = wb.active
    
    tickets = []
    release_date = None
    
    for row in range(2, ws.max_row + 1):
        ticket = {
            "id": ws.cell(row=row, column=1).value,
            "title": ws.cell(row=row, column=2).value,
            "description": ws.cell(row=row, column=3).value,
            "release_date": ws.cell(row=row, column=4).value,
            "story_owner": ws.cell(row=row, column=5).value,
            "area": ws.cell(row=row, column=6).value,
            "customer": ws.cell(row=row, column=7).value,
            "ticket_url": ws.cell(row=row, column=8).value,
            "release_note": ws.cell(row=row, column=9).value,
            "release_notes_summary": ws.cell(row=row, column=10).value,
        }
        tickets.append(ticket)
        if not release_date and ticket["release_date"]:
            release_date = str(ticket["release_date"])
    
    return {"status": "success", "release_date": release_date, "tickets": tickets}


async def format_confluence_content(release_date: str, tickets: List[dict]) -> dict:
    """
    Format release notes data as Markdown for Confluence publishing.
    
    Creates a Markdown table with Area, Summary, and Details columns.
    The Atlassian MCP tool converts this Markdown to Confluence's internal
    Atlassian Document Format (ADF) automatically.
    
    Args:
        release_date: The release date string for the page title
        tickets: List of ticket dictionaries from read_excel_data()
    
    Returns:
        dict: Contains status, formatted Markdown content, and suggested page title
    """
    
    # Markdown content
    lines = []
    
    # Intro
    lines.append("This release includes the following updates and improvements.")
    lines.append("")
    
    # Table header
    lines.append("| Area | Summary | Details |")
    lines.append("|------|---------|---------|")
    
    # Table rows
    for ticket in tickets:
        area_path = ticket.get("area", "")
        area = area_path.split("\\")[-1] if area_path else ""
        
        summary = ticket.get("release_notes_summary", "").replace("|", "\\|")  # CHANGED
        release_note = ticket.get("release_note", "").replace("|", "\\|")
        
        lines.append(f"| {area} | {summary} | {release_note} |")
    
    content = "\n".join(lines)
    
    return {
        "status": "success",
        "content": content,
        "page_title": f"Release Notes - {release_date}"
    }

### Agents

**ticket_processor_agent:**
    Connects to the Azure DevOps MCP server, fetches work items, generates
    user‑friendly release notes, and writes all processed ticket data into
    a structured Excel file.

**confluence_publisher_agent:**
    Loads the generated Excel data, formats it into Confluence‑ready Markdown,
    connects to the Atlassian MCP server, and publishes the final release
    notes page into the correct Confluence space.

These two agents are orchestrated by SequentialAgent, which runs the workflow
sequentially: first generating release notes, then publishing them.


In [12]:
ticket_processor_agent = LlmAgent(
    model=MODEL_NAME,
    name="TicketProcessorAgent",
    description="Fetches tickets from Azure DevOps and creates release notes",
    instruction=f"""
        You are a Release Notes Creator.
        You have the following tools which are all you need to complete your task:
        
        * mcp_ado_wit_get_query_results_by_id – (Azure DevOps MCP tool) Retrieves all work item IDs used to generate release notes.
        * initialize_excel – (local tool) Creates a new Excel file with the required release‑notes structure.
        * append_ticket_to_excel – (local tool) Saves each processed ticket and its generated release note into the Excel file.

        Before proceeding with the steps you must connect to Azure DevOps MCP Server!

        **STEP 1: FETCH TICKET IDs FROM AZURE DEVOPS**
        
        Call the MCP tool `mcp_ado_wit_get_query_results_by_id` with these EXACT parameters:
        {{
          "id": "{AZURE_DEVOPS_QUERY_ID}",
          "project": "{AZURE_DEVOPS_PROJECT}",
          "organization": "{AZURE_DEVOPS_ORG}"
        }}
        
        Extract all work item IDs from the result.
        
        **STEP 2: INITIALIZE EXCEL FILE**
        
        Call `initialize_excel()` to create the Excel file.
        Store the returned file_path for subsequent calls.
        
        **STEP 3: PROCESS EACH TICKET (ONE BY ONE)**
        
        For EACH work item ID, do the following steps IMMEDIATELY before moving to the next ticket:
        
        a) Call `mcp_ado_wit_get_work_item` to get the full details
        b) Extract these fields:
           - id: System.Id(system field: work item ID)
           - title: System.Title field
           - description: System.Description field (full text)
           - release_date: Custom.ReleaseDate or similar field
           - story_owner: Custom.StoryOwner or similar field
           - area: System.AreaPath field
           - customer: Custom.Customer field (if available)
           - ticket_url: https://dev.azure.com/{AZURE_DEVOPS_ORG}/{AZURE_DEVOPS_PROJECT}/_workitems/edit/[ID]
        
        c) Generate a release note from the description using these guidelines:
        
        {RELEASE_NOTES_INSTRUCTIONS}

        d) Generate a release_notes_summary: Create a short, user-friendly but meaningful title (5-10 words max) 
           that summarizes what changed. This should be suitable for publishing in release notes.
           Examples:
           - "Faster profile page loading"
           - "New asset tree creation in Censhare"
           - "Improved Censhare domain name display"
        
        e) Call `append_ticket_to_excel` with ALL the extracted fields AND the generated release_note.
           This saves the ticket to Excel IMMEDIATELY.
        
        f) Move to the next ticket ID and repeat steps a-e.
        
        **STEP 4: OUTPUT**
        
        After processing all tickets, return the Excel file path in your response.
        """,
    tools=[
        McpToolset(
            connection_params=StdioConnectionParams(
                server_params=StdioServerParameters(
                    command="npx",
                    # args=["-y", "@azure-devops/mcp", AZURE_DEVOPS_ORG, "-d", "work-items"],
                    args=["-y", "@azure-devops/mcp", AZURE_DEVOPS_ORG],
                ),
                timeout=120,
            ),
            # tool_filter=[
            #     'mcp_ado_wit_get_query_results_by_id', 
            # ]
        ),
        
        initialize_excel,
        append_ticket_to_excel,
    ],
    output_key="excel_file_path",
)

confluence_publisher_agent = LlmAgent(
    model=MODEL_NAME,
    name="ConfluencePublisherAgent",
    description="Publishes release notes to Confluence",
    instruction="""
        You are a Confluence Publisher working in the marmind-knowledgebase.atlassian.net Confluence!

        You have the following tools which are all you need to complete your task:
        
        * read_excel_data – (local tool) Loads all stored ticket data and release notes from the Excel file.
        * format_confluence_content – (local tool) Converts the loaded ticket data into clean, publish‑ready Markdown.
        * createConfluencePage – (Atlassian MCP tool) Creates a new Confluence page in the correct space and parent page.

        Before proceeding with the steps you must connect to Atlassian MCP Server!

        
        **STEP 1: READ EXCEL FILE**
        
        The Excel file path is available at: {excel_file_path}
        
        Call `read_excel_data` with this file path to get all tickets and release notes.
        
        **STEP 2: FORMAT CONFLUENCE CONTENT**
        
        Call `format_confluence_content` with:
        - release_date: from the Excel data
        - tickets: the list of tickets from the Excel data
        
        
        **STEP 3: CREATE CONFLUENCE PAGE**
        
        Use `createConfluencePage` MCP tool with these parameters:
        - spaceId: "327684"
        - parentPageId: "12746753"
        - title: Use the page_title from step 2
        - body: Use the content from step 2 (this should be Markdown format)
        
        **STEP 4: CONFIRM SUCCESS**
        
        Report the URL of the created Confluence page.
        Follow this format: [Page Title](Link)
        """,
    tools=[
        read_excel_data,
        format_confluence_content,
        
        McpToolset(
            connection_params=StdioConnectionParams(
                server_params=StdioServerParameters(
                    command='npx',
                    args=[
                        '-y',
                        'mcp-remote',
                        'https://mcp.atlassian.com/v1/sse'
                    ],
                ),
                timeout=120 
            ),
            # tool_filter=[
            #     'createConfluencePage', 
            # ]
        ),
        
    ],
    output_key="confluence_page_url",
)

root_agent = SequentialAgent(
    name="ReleaseNotesWorkflow",
    description="Creates release notes from Azure DevOps and publishes to Confluence",
    sub_agents=[
        ticket_processor_agent, 
        confluence_publisher_agent
    ],
)

### Runner

In [13]:
session_service = InMemorySessionService()
runner = Runner(agent=root_agent, app_name="release_notes_agent", session_service=session_service)

USER_ID = "user"
SESSION_ID = "release_notes_session"
session_initialized = False


async def chat():
    """Interactive chat with the agent."""
    global session_initialized
    
    if not session_initialized:
        await session_service.create_session(
            app_name="release_notes_agent",
            user_id=USER_ID,
            session_id=SESSION_ID,
        )
        session_initialized = True
        print("🚀 Release Notes Agent ready!")
        print("Example: 'Create release notes and publish to Confluence'")
        print("Type 'quit', 'exit', 'stop' to exit.\n")
    
    while True:
        user_input = input("You: ").strip()
        
        if user_input.lower() in ["quit", "exit", "stop"]:
            print("👋 Goodbye!")
            break
        
        if not user_input:
            continue
        
        message = types.Content(role="user", parts=[types.Part(text=user_input)])
        print("Agent: ", end="", flush=True)
        
        # Suppress warnings and stderr output
        with open(os.devnull, 'w') as f, contextlib.redirect_stderr(f), warnings.catch_warnings():
            warnings.simplefilter("ignore")
            async for event in runner.run_async(
                user_id=USER_ID,
                session_id=SESSION_ID,
                new_message=message
            ):
                if event.content and event.content.parts:
                    for part in event.content.parts:
                        if part.text:
                            print(part.text, end="", flush=True)
        
        print("\n")

In [14]:
await chat()

🚀 Release Notes Agent ready!
Example: 'Create release notes and publish to Confluence'
Type 'quit', 'exit', 'stop' to exit.



You:  Create release notes and publish to Confluence


Agent: The release notes have been successfully generated and saved.

**Excel File Path:** `./output/release_notes_2026-01-03.xlsx`

### Summary of Processed Tickets:

1.  **Ticket 54068**: Improved Censhare domain name display
    *   **Release Note**: Updated: The Censhare domain field now displays user-friendly display names instead of technical identifiers, improving readability in the asset details view.
2.  **Ticket 55981**: New asset tree creation in Censhare
    *   **Release Note**: Added: Users can now create tree objects directly in Censhare from the Marmind object header, facilitating immediate asset management without prior assignments.
3.  **Ticket 57533**: Asset-type-specific metadata synchronization
    *   **Release Note**: Added: Metadata synchronization now supports asset-type-specific field mappings, ensuring that different Censhare asset types correctly sync with their corresponding Marmind custom fields.The release notes have been successfully published to Conflue

You:  quit


👋 Goodbye!


In [3]:
import google.adk
print(google.adk.__version__)

1.21.0
